# Send Notification to Slack

### mlconfig

In [1]:
from mlrun import mlconf
import os

mlconf.dbpath = 'http://mlrun-api:8080'
mlconf.artifact_path = mlconf.artifact_path or f'{os.environ["HOME"]}/artifacts'

### save

In [2]:
from mlrun import code_to_function
import yaml

with open("item.yaml") as item_file:
    items = yaml.load(item_file, Loader=yaml.FullLoader)
    
# create job function object from notebook code
fn = code_to_function(
    items["name"],
    handler=items["spec"]["handler"],
    kind=items["spec"]["kind"],
    filename=items["spec"]["filename"],
    description=items["description"],
    categories=items["categories"],
    labels=items["labels"],
)
fn.export("slack_notify.yaml")

> 2021-02-17 11:20:58,889 [info] function spec saved to path: slack_notify.yaml


## examples

In [3]:
from mlrun import import_function
func = import_function("hub://slack_notify")

In [4]:
from mlrun import NewTask, run_local
from slack_notify import slack_notify

#Slack incoming webhook URL. Please read: https://api.slack.com/messaging/webhooks
task_params = {
    "webhook_url" : "https://hooks.slack.com/services/xxxxxxxx/xxxxxxxxx/xxxxxxxxxxxxxx",
    "notification_text" : "Test Notification"
}

In [5]:
task = NewTask(
    name="tasks-slack-notify", 
    params = task_params,
    handler=slack_notify)

### run local where artifact path is fixed 

In [6]:
run = run_local(task, artifact_path=mlconf.artifact_path)

> 2021-02-17 11:20:59,044 [info] starting run tasks-slack-notify uid=bb4405e6f9d84e9bbe1a881bf6708ebe DB=http://mlrun-api:8080
==== https://hooks.slack.com/services/xxxxxxxx/xxxxxxxxx/xxxxxxxxxxxxxx
Response: no_team
Response code: 404


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...f6708ebe,0,Feb 17 11:20:59,completed,tasks-slack-notify,v3io_user=eyalskind=handlerowner=eyalshost=jupyter-eyals-666bf556fc-5v7bf,,webhook_url=https://hooks.slack.com/services/xxxxxxxx/xxxxxxxxx/xxxxxxxxxxxxxxnotification_text=Test Notification,,


to track results use .show() or .logs() or in CLI: 
!mlrun get run bb4405e6f9d84e9bbe1a881bf6708ebe --project default , !mlrun logs bb4405e6f9d84e9bbe1a881bf6708ebe --project default
> 2021-02-17 11:20:59,257 [info] run executed, status=completed


### run remote where artifact path includes the run id

In [7]:
func.deploy()

> 2021-02-17 11:20:59,261 [info] starting remote build, image: .mlrun/func-default-slack-notify-latest
INFO[0020] Retrieving image manifest mlrun/mlrun:0.6.0  
INFO[0020] Retrieving image manifest mlrun/mlrun:0.6.0  
INFO[0020] Built cross stage deps: map[]                
INFO[0020] Retrieving image manifest mlrun/mlrun:0.6.0  
INFO[0021] Retrieving image manifest mlrun/mlrun:0.6.0  
INFO[0021] Executing 0 build triggers                   
INFO[0021] Unpacking rootfs as cmd RUN pip install requests requires it. 
INFO[0035] RUN pip install requests                     
INFO[0035] Taking snapshot of full filesystem...        
INFO[0047] cmd: /bin/sh                                 
INFO[0047] args: [-c pip install requests]              
INFO[0047] Running: [/bin/sh -c pip install requests]   
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
INFO[0048] Taking snapshot of full filesystem...        


True

In [8]:
func.run(task, params=task_params,  workdir=mlconf.artifact_path)

> 2021-02-17 11:21:53,125 [info] starting run tasks-slack-notify uid=a993d371128a4b89a215ca869ddb9fdb DB=http://mlrun-api:8080
> 2021-02-17 11:21:53,260 [info] Job is running in the background, pod: tasks-slack-notify-n9c22
==== https://hooks.slack.com/services/xxxxxxxx/xxxxxxxxx/xxxxxxxxxxxxxx
Response: no_team
Response code: 404
> 2021-02-17 11:21:56,015 [info] run executed, status=completed
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...9ddb9fdb,0,Feb 17 11:21:55,completed,tasks-slack-notify,v3io_user=eyalskind=jobowner=eyalshost=tasks-slack-notify-n9c22,,webhook_url=https://hooks.slack.com/services/xxxxxxxx/xxxxxxxxx/xxxxxxxxxxxxxxnotification_text=Test Notification,,


to track results use .show() or .logs() or in CLI: 
!mlrun get run a993d371128a4b89a215ca869ddb9fdb --project default , !mlrun logs a993d371128a4b89a215ca869ddb9fdb --project default
> 2021-02-17 11:21:56,358 [info] run executed, status=completed


In [9]:
func.doc()

function: slack-notify
Send Slack notification
default handler: slack_notify
entry points:
  slack_notify: Summarize a table
    context(MLClientCtx)  - the function context
    webhook_url(str)  - Slack incoming webhook URL. Please read: https://api.slack.com/messaging/webhooks, default=URL
    slack_blocks(List[str])  - Message blocks list. NOT IMPLEMENTED YET
    notification_text(str)  - Notification text, default=Notification
